In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [2]:
# Import our input dataset
read_file = pd.read_excel ("C:/Users/Admin/group9/Resources/RealEstData05132023.xlsx")
read_file.to_csv ("C:/Users/Admin/group9/Resources/RealEstData05132023.csv", 
                  index=None,
                  header=True)

wake_df = pd.DataFrame(pd.read_csv("C:/Users/Admin/group9/Resources/RealEstData05132023.csv"))
wake_df.head(10)

C:\Users\Admin\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (12,47,70,71,72,79) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,OWNER1,OWNER2,Mailing_address1,Mailing_Address2,Mailing_Address3,REAL_ESTATE_ID,CARD_NUMBER,NUMBER_OF_CARDS,Street_Number,Street_Prefix,...,Land_Deferred_code,Land_Deferred_Amount,Historic_Deferred_code,Historic_Deferred_Amount,RECYCLED_UNITS,Disq_and_Qual_flag,Land_Disq_and_Qual_flag,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
0,NORTHGATE PROPERTIES LLC,NaN,8465 LENTIC CT,RALEIGH NC 27615-4964,NaN,1,1,1,1506,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27604.0
1,BONES 13 LLC,NaN,306 MIDENHALL WAY,CARY NC 27513-5569,NaN,3,1,2,6012,NaN,...,NaN,0,NaN,0,0,A,NaN,20.0,RALEIGH,27617.0
2,BONES 13 LLC,NaN,306 MIDENHALL WAY,CARY NC 27513-5569,NaN,3,2,2,6012,NaN,...,NaN,0,NaN,0,0,A,NaN,34.0,RALEIGH,27617.0
3,HARRISPARK PROPERTIES INC,NaN,608 GASTON ST STE 200,RALEIGH NC 27603-1258,NaN,4,1,1,1601,NaN,...,NaN,0,NaN,0,0,T,NaN,85.0,RALEIGH,27604.0
4,"MOGHADASS, INC",NaN,1532 PROVIDENCE RD S STE 230,WAXHAW NC 28173-8313,NaN,5,1,1,1831,NaN,...,NaN,0,NaN,0,0,A,NaN,200.0,RALEIGH,27604.0
5,RALEIGH CITY OF,NaN,PO BOX 590,RALEIGH NC 27602-0590,NaN,16,1,1,609,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27610.0
6,SKS PROPERTIES,NaN,PO BOX 19808,RALEIGH NC 27619-9808,NaN,17,1,1,103,NaN,...,NaN,0,NaN,0,0,NaN,NaN,NaN,RALEIGH,27607.0
7,N C BEER WHOLESALERS ASSOC,NaN,210 N PERSON ST,RALEIGH NC 27601-1047,NaN,18,1,1,210,N,...,NaN,0,NaN,0,0,A,NaN,35.0,RALEIGH,27601.0
8,"MICKENS, ROBERT EARL & ANNIE M",NaN,2457 BERTIE DR,RALEIGH NC 27610-1730,NaN,19,1,1,2457,NaN,...,NaN,0,NaN,0,1,A,NaN,1.0,RALEIGH,27610.0
9,"COREY, JAMES CURTIS & REGINA B",NaN,2848 PROVIDENCE RD,RALEIGH NC 27610-5210,NaN,20,1,1,2848,NaN,...,NaN,0,NaN,0,1,A,NaN,1.0,RALEIGH,27610.0


In [3]:
# Export just the head(10) of our data as a reference on our ReadMe.md
#wake_head_df = wake_df.head(10)
#wake_head_df.to_csv(r'C:/Users/Admin/group9/Resources/wake_head_df.csv', index = False, header=True)

In [4]:
# Seeing how many null values we have
wake_df.isnull().sum()

OWNER1                          0
OWNER2                     359944
Mailing_address1                0
Mailing_Address2               14
Mailing_Address3           414218
                            ...  
Disq_and_Qual_flag          82832
Land_Disq_and_Qual_flag    281472
TYPE_AND_USE                54016
PHYSICAL_CITY                2431
PHYSICAL_ZIP_CODE            4290
Length: 87, dtype: int64

In [6]:
# Drop all columns that aren't relevant
cleaner_wake_df = wake_df[['REAL_ESTATE_ID', 'Township','Deeded_Acreage', 'Total_sale_Price', 'Total_Sale_Date' 
                           , 'Deed_Date','Year_Built', 'HEATED_AREA', 'DESIGN_STYLE','BATH','Story_Height','TYPE_AND_USE',
                           'PHYSICAL_CITY','PHYSICAL_ZIP_CODE']]
cleaner_wake_df.head()
# Export cleaner_wake_df for use in README.md
#cleaner_wake_df.to_csv(r'C:/Users/Admin/group9/Resources/cleaner_wake_df.csv', index = False, header=True)

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Deed_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
0,1,1,0.32,0,NaN,2020-09-30 00:00:00,0,0,NaN,NaN,NaN,NaN,RALEIGH,27604.0
1,3,8,2.00,740000,2012-03-05,2012-03-05 00:00:00,1979,4500,A,I,A,20.0,RALEIGH,27617.0
2,3,8,2.00,740000,2012-03-05,2012-03-05 00:00:00,1989,4008,A,I,A,34.0,RALEIGH,27617.0
3,4,1,1.38,185000,1984-10-08,2011-01-19 00:00:00,1993,8500,A,I,A,85.0,RALEIGH,27604.0
4,5,1,1.11,750000,2016-06-17,2016-06-17 00:00:00,1968,3088,A,I,A,200.0,RALEIGH,27604.0


In [7]:
# Checking datatypes
cleaner_wake_df.dtypes

REAL_ESTATE_ID         int64
Township               int64
Deeded_Acreage       float64
Total_sale_Price       int64
Total_Sale_Date       object
Deed_Date             object
Year_Built             int64
HEATED_AREA            int64
DESIGN_STYLE          object
BATH                  object
Story_Height          object
TYPE_AND_USE         float64
PHYSICAL_CITY         object
PHYSICAL_ZIP_CODE    float64
dtype: object

In [8]:
# Checking dataset size
cleaner_wake_df.shape

(439030, 14)

In [9]:
# Seeing how many null values we have
cleaner_wake_df.isnull().sum()

REAL_ESTATE_ID           0
Township                 0
Deeded_Acreage           0
Total_sale_Price         0
Total_Sale_Date      82576
Deed_Date                0
Year_Built               0
HEATED_AREA              0
DESIGN_STYLE         56351
BATH                 56311
Story_Height         56116
TYPE_AND_USE         54016
PHYSICAL_CITY         2431
PHYSICAL_ZIP_CODE     4290
dtype: int64

In [10]:
# Dropping null values
cleaner_wake_df = cleaner_wake_df.dropna()
cleaner_wake_df.head(10)

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Deed_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
1,3,8,2.00,740000,2012-03-05,2012-03-05 00:00:00,1979,4500,A,I,A,20.0,RALEIGH,27617.0
2,3,8,2.00,740000,2012-03-05,2012-03-05 00:00:00,1989,4008,A,I,A,34.0,RALEIGH,27617.0
3,4,1,1.38,185000,1984-10-08,2011-01-19 00:00:00,1993,8500,A,I,A,85.0,RALEIGH,27604.0
4,5,1,1.11,750000,2016-06-17,2016-06-17 00:00:00,1968,3088,A,I,A,200.0,RALEIGH,27604.0
7,18,1,0.40,82000,1983-01-10,1983-01-10 00:00:00,1984,3341,A,I,C,35.0,RALEIGH,27601.0
8,19,1,0.21,34500,1974-01-01,1974-01-01 00:00:00,1964,1828,I,C,A,1.0,RALEIGH,27610.0
9,20,16,0.46,35500,1983-05-18,1993-10-28 00:00:00,1970,1240,A,A,A,1.0,RALEIGH,27610.0
11,22,10,0.46,37500,2004-09-16,2004-09-16 00:00:00,1900,2261,A,C,A,1.0,WENDELL,27591.0
12,23,1,0.79,210000,2018-06-27,2018-06-27 00:00:00,1916,2440,E,J,C,2.0,RALEIGH,27606.0
14,25,8,0.96,70000,1971-01-01,1971-01-01 00:00:00,1971,3770,A,J,A,1.0,RALEIGH,27613.0


In [11]:
# Change float64 values where we don't need decimals to int64
cleaner_wake_df['PHYSICAL_ZIP_CODE'] = cleaner_wake_df['PHYSICAL_ZIP_CODE'].apply(str)
cleaner_wake_df['TYPE_AND_USE'] = cleaner_wake_df['TYPE_AND_USE'].apply(int)

In [18]:
# Dropping all building types that aren't single family
cleaner_wake_df.drop(cleaner_wake_df[cleaner_wake_df['TYPE_AND_USE'] != 1].index, inplace = True)
cleaner_wake_df.head()

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Deed_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,TYPE_AND_USE,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
8,19,1,0.21,34500,1974-01-01,1974-01-01 00:00:00,1964,1828,I,C,A,1,RALEIGH,27610.0
9,20,16,0.46,35500,1983-05-18,1993-10-28 00:00:00,1970,1240,A,A,A,1,RALEIGH,27610.0
11,22,10,0.46,37500,2004-09-16,2004-09-16 00:00:00,1900,2261,A,C,A,1,WENDELL,27591.0
14,25,8,0.96,70000,1971-01-01,1971-01-01 00:00:00,1971,3770,A,J,A,1,RALEIGH,27613.0
16,30,1,0.47,380000,2015-08-12,2015-08-12 00:00:00,1946,1789,A,C,A,1,RALEIGH,27607.0


In [19]:
# Confirming that we only have TYPE_AND_USE as 1
cleaner_wake_df['TYPE_AND_USE'].unique()

array([1], dtype=int64)

In [20]:
# Checking datatypes
cleaner_wake_df.dtypes

REAL_ESTATE_ID         int64
Township               int64
Deeded_Acreage       float64
Total_sale_Price       int64
Total_Sale_Date       object
Deed_Date             object
Year_Built             int64
HEATED_AREA            int64
DESIGN_STYLE          object
BATH                  object
Story_Height          object
TYPE_AND_USE           int64
PHYSICAL_CITY         object
PHYSICAL_ZIP_CODE     object
dtype: object

In [21]:
# Checking dataset shape
cleaner_wake_df.shape

(323414, 14)

In [22]:
# Seeing how many null values we have
cleaner_wake_df.isnull().sum()

REAL_ESTATE_ID       0
Township             0
Deeded_Acreage       0
Total_sale_Price     0
Total_Sale_Date      0
Deed_Date            0
Year_Built           0
HEATED_AREA          0
DESIGN_STYLE         0
BATH                 0
Story_Height         0
TYPE_AND_USE         0
PHYSICAL_CITY        0
PHYSICAL_ZIP_CODE    0
dtype: int64

In [23]:
# Dropping values we no longer need/are redundant
cleaned_wake_df = cleaner_wake_df.drop(['Deed_Date', 'TYPE_AND_USE'], axis=1)
cleaned_wake_df.head()

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,PHYSICAL_CITY,PHYSICAL_ZIP_CODE
8,19,1,0.21,34500,1974-01-01,1964,1828,I,C,A,RALEIGH,27610.0
9,20,16,0.46,35500,1983-05-18,1970,1240,A,A,A,RALEIGH,27610.0
11,22,10,0.46,37500,2004-09-16,1900,2261,A,C,A,WENDELL,27591.0
14,25,8,0.96,70000,1971-01-01,1971,3770,A,J,A,RALEIGH,27613.0
16,30,1,0.47,380000,2015-08-12,1946,1789,A,C,A,RALEIGH,27607.0


In [24]:
# Checking datatypes
cleaned_wake_df.dtypes

REAL_ESTATE_ID         int64
Township               int64
Deeded_Acreage       float64
Total_sale_Price       int64
Total_Sale_Date       object
Year_Built             int64
HEATED_AREA            int64
DESIGN_STYLE          object
BATH                  object
Story_Height          object
PHYSICAL_CITY         object
PHYSICAL_ZIP_CODE     object
dtype: object

In [28]:
# Converting Total_Sale_Date to datetime format
cleaned_wake_df['Total_Sale_Date'] = pd.to_datetime(cleaned_wake_df['Total_Sale_Date'], format ="%Y-%m-%d")
# Checking to see if it worked
cleaned_wake_df.dtypes

REAL_ESTATE_ID                int64
Township                      int64
Deeded_Acreage              float64
Total_sale_Price              int64
Total_Sale_Date      datetime64[ns]
Year_Built                    int64
HEATED_AREA                   int64
DESIGN_STYLE                 object
BATH                         object
Story_Height                 object
PHYSICAL_CITY                object
PHYSICAL_ZIP_CODE            object
dtype: object

In [32]:
# Seperating Total_Sale_Date into y m d columns
cleaned_wake_df['year']= cleaned_wake_df['Total_Sale_Date'].dt.year
cleaned_wake_df['month']= cleaned_wake_df['Total_Sale_Date'].dt.month
cleaned_wake_df['day']= cleaned_wake_df['Total_Sale_Date'].dt.day

cleaned_wake_df.dtypes

REAL_ESTATE_ID                int64
Township                      int64
Deeded_Acreage              float64
Total_sale_Price              int64
Total_Sale_Date      datetime64[ns]
Year_Built                    int64
HEATED_AREA                   int64
DESIGN_STYLE                 object
BATH                         object
Story_Height                 object
PHYSICAL_CITY                object
PHYSICAL_ZIP_CODE            object
year                          int64
month                         int64
day                           int64
dtype: object

In [42]:
# Dropping values we no longer need/are redundant
cleaned_wake_df.drop(['day'], axis=1, inplace=True)
cleaned_wake_df.head()

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,year,month
8,19,1,0.21,34500,1974-01-01,1964,1828,I,C,A,RALEIGH,27610.0,1974,1
9,20,16,0.46,35500,1983-05-18,1970,1240,A,A,A,RALEIGH,27610.0,1983,5
11,22,10,0.46,37500,2004-09-16,1900,2261,A,C,A,WENDELL,27591.0,2004,9
14,25,8,0.96,70000,1971-01-01,1971,3770,A,J,A,RALEIGH,27613.0,1971,1
16,30,1,0.47,380000,2015-08-12,1946,1789,A,C,A,RALEIGH,27607.0,2015,8


In [44]:
# Creating a "Quarter" column
cleaned_wake_df['Quarter'] = pd.cut(cleaned_wake_df['month'], [0,3,6,9,12], labels = [1,2,3,4])
cleaned_wake_df.head(10)

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,year,month,Quarter
8,19,1,0.21,34500,1974-01-01,1964,1828,I,C,A,RALEIGH,27610.0,1974,1,1
9,20,16,0.46,35500,1983-05-18,1970,1240,A,A,A,RALEIGH,27610.0,1983,5,2
11,22,10,0.46,37500,2004-09-16,1900,2261,A,C,A,WENDELL,27591.0,2004,9,3
14,25,8,0.96,70000,1971-01-01,1971,3770,A,J,A,RALEIGH,27613.0,1971,1,1
16,30,1,0.47,380000,2015-08-12,1946,1789,A,C,A,RALEIGH,27607.0,2015,8,3
17,31,13,0.39,337500,2002-07-17,1970,2463,A,C,A,RALEIGH,27615.0,2002,7,3
19,33,4,0.50,32500,1979-02-06,1969,2389,G,E,A,RALEIGH,27607.0,1979,2,1
20,34,1,0.70,19500,1968-01-01,1968,1280,A,C,A,RALEIGH,27606.0,1968,1,1
21,35,1,0.49,319000,1996-01-16,1938,4650,A,F,B,RALEIGH,27608.0,1996,1,1
24,37,1,0.13,425000,2008-04-15,1921,1890,A,C,C,RALEIGH,27604.0,2008,4,2


In [39]:
# Checking dataset shape
cleaned_wake_df.shape

(323414, 15)

In [45]:
cleaned_wake_df.dtypes

REAL_ESTATE_ID                int64
Township                      int64
Deeded_Acreage              float64
Total_sale_Price              int64
Total_Sale_Date      datetime64[ns]
Year_Built                    int64
HEATED_AREA                   int64
DESIGN_STYLE                 object
BATH                         object
Story_Height                 object
PHYSICAL_CITY                object
PHYSICAL_ZIP_CODE            object
year                          int64
month                         int64
Quarter                    category
dtype: object

In [46]:
# Dropping index
cleaned_wake_df = cleaned_wake_df.reset_index(drop=True)
cleaned_wake_df.head(10)

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Total_Sale_Date,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,year,month,Quarter
0,19,1,0.21,34500,1974-01-01,1964,1828,I,C,A,RALEIGH,27610.0,1974,1,1
1,20,16,0.46,35500,1983-05-18,1970,1240,A,A,A,RALEIGH,27610.0,1983,5,2
2,22,10,0.46,37500,2004-09-16,1900,2261,A,C,A,WENDELL,27591.0,2004,9,3
3,25,8,0.96,70000,1971-01-01,1971,3770,A,J,A,RALEIGH,27613.0,1971,1,1
4,30,1,0.47,380000,2015-08-12,1946,1789,A,C,A,RALEIGH,27607.0,2015,8,3
5,31,13,0.39,337500,2002-07-17,1970,2463,A,C,A,RALEIGH,27615.0,2002,7,3
6,33,4,0.50,32500,1979-02-06,1969,2389,G,E,A,RALEIGH,27607.0,1979,2,1
7,34,1,0.70,19500,1968-01-01,1968,1280,A,C,A,RALEIGH,27606.0,1968,1,1
8,35,1,0.49,319000,1996-01-16,1938,4650,A,F,B,RALEIGH,27608.0,1996,1,1
9,37,1,0.13,425000,2008-04-15,1921,1890,A,C,C,RALEIGH,27604.0,2008,4,2


In [47]:
# Export version of dataset with 
cleaned_wake_df.to_csv(r'C:/Users/Admin/group9/Resources/cleaned_wake_w_date_df.csv', index = False, header=True)

In [49]:
# Dropping values we no longer need/are redundant
cleaned_wake_df.drop(['Total_Sale_Date'], axis=1, inplace=True)
cleaned_wake_df.head()

,REAL_ESTATE_ID,Township,Deeded_Acreage,Total_sale_Price,Year_Built,HEATED_AREA,DESIGN_STYLE,BATH,Story_Height,PHYSICAL_CITY,PHYSICAL_ZIP_CODE,year,month,Quarter
0,19,1,0.21,34500,1964,1828,I,C,A,RALEIGH,27610.0,1974,1,1
1,20,16,0.46,35500,1970,1240,A,A,A,RALEIGH,27610.0,1983,5,2
2,22,10,0.46,37500,1900,2261,A,C,A,WENDELL,27591.0,2004,9,3
3,25,8,0.96,70000,1971,3770,A,J,A,RALEIGH,27613.0,1971,1,1
4,30,1,0.47,380000,1946,1789,A,C,A,RALEIGH,27607.0,2015,8,3


In [50]:
cleaned_wake_df.dtypes

REAL_ESTATE_ID          int64
Township                int64
Deeded_Acreage        float64
Total_sale_Price        int64
Year_Built              int64
HEATED_AREA             int64
DESIGN_STYLE           object
BATH                   object
Story_Height           object
PHYSICAL_CITY          object
PHYSICAL_ZIP_CODE      object
year                    int64
month                   int64
Quarter              category
dtype: object

In [51]:
# Export just cleaned_wake_df
cleaned_wake_df.to_csv(r'C:/Users/Admin/group9/Resources/cleaned_wake_df.csv', index = False, header=True)